In [1]:
	
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import urllib.parse
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import PaymentReferenceSearch, AccessAccountsReceivable, AccessPayments
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from typing import Union

In [2]:
# Langchain API KEY
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
# DATABASE Connection settings
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

# Creating conneciton to database
conn = psycopg2.connect(f"dbname={DB_NAME} user={USERNAME} password={PASSWORD}")
accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
payments= pd.read_sql("SELECT * FROM payments", conn)

/var/folders/4s/q4zndhns13bdmf7q1_8p3crc0000gn/T/ipykernel_17482/3319540732.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
/var/folders/4s/q4zndhns13bdmf7q1_8p3crc0000gn/T/ipykernel_17482/3319540732.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments= pd.read_sql("SELECT * FROM payments", conn)


In [3]:
class Context():
    """Custom runtime context schema."""
    user_id: str

class ARResponse(BaseModel):
    """Accounts Receivable dataframe content"""
    invoice_number: int 
    date: dt.date
    customer_name: str
    customer_number: int
    amount: float 
    due_date: dt.date 
    payment: float | None = None
    payment_date: dt.date | None = None
    payment_id: int | None = None

class PayResponse(BaseModel):
    "Payments dataframe content"
    transaction_id: int
    payment_date: dt.date 
    payment_amount: float 
    payment_reference: str | None = None
    matched: bool

In [4]:
prompt =f"""You are an expert accountant in the accounts receivables department. You have access to the following tools:
            - AccessAccountsReceivable to retrieve the current accounts receivables data.
            - AccessPayments to retrieve the payments received.
            - PaymentReferenceSearch to match the payments received to the accounts receivables, returning an updated accounts receivable dataframe and and updated payments dataframe.
            If a user asks you to show the accounts receivables, payments data, or to update accounts receivables, make sure you use these tools.
            Be concise and accurate.
            """

In [5]:
# Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.
checkpointer = InMemorySaver()

In [6]:
tools=[PaymentReferenceSearch, AccessAccountsReceivable, AccessPayments]
model = ChatOllama(
    model="ministral-3:14b",
    temperature=0,
) 
# model_with_tools = model.bind_tools(tools=tools) # only for OpenAI
#agent = create_agent(model=model, system_prompt=prompt, tools=tools, response_format=ToolStrategy(Union[ARResponse, PayResponse]), checkpointer=checkpointer)
agent = create_agent(model=model, system_prompt=prompt, tools=tools, checkpointer=checkpointer)


In [ ]:
# Run agent
#`thread_id` is a unique identifier for a given conversation
config = {"configurable": {"thread_id": "20"}}

response = agent.invoke(
    {"messages": [{"role": "user", 
                   "content": """ Without responding with questions and using the tools provided perform the following steps in order:
                   1. Access the accounts receivables as a pandas dataframe,
                   2. Access the payments received as a pandas dataframe,
                   3. Match the payments received to the open accounts receivables using the tool provided,
                   4. Update the accounts receivables as a pandas dataframe,
                   Return the updated accounts receivable dataframe for me to review."""}]},
    config=config,
    context=Context()
)

/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments = pd.read_sql("SELECT * FROM payments", conn)


ResponseError: unexpected end of JSON input (status code: 500)